In [ ]:
import cv2
import numpy as np
from deepface import DeepFace
import os
import sqlite3
from datetime import datetime

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect("attendance.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS attendance_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            date TEXT NOT NULL,
            time TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

# Function to register a new face
def register_face(name, frame):
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    # Save the captured image
    image_path = f"{dataset_path}/{name}.jpg"
    cv2.imwrite(image_path, frame)
    print(f"Registered {name} at {image_path}")

# Function to recognize face and log attendance in SQLite
def recognize_and_log(frame):
    try:
        # Use DeepFace to find the closest match in the dataset
        result = DeepFace.find(img_path=frame, db_path=dataset_path, model_name="Facenet", detector_backend="opencv", distance_metric="cosine")
        
        # Check if a match is found
        if len(result) > 0 and not result[0].empty:
            identity = result[0]["identity"][0]
            name = os.path.splitext(os.path.basename(identity))[0]
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            date, time = current_time.split(" ")
            
            # Log attendance in SQLite
            conn = sqlite3.connect("attendance.db")
            cursor = conn.cursor()
            cursor.execute("INSERT INTO attendance_log (name, date, time) VALUES (?, ?, ?)", (name, date, time))
            conn.commit()
            conn.close()
            return f"Attendance marked for {name} at {current_time}"
        else:
            return "Face not recognized"
    except Exception as e:
        return f"Error: {str(e)}"

# Function to view attendance log
def view_attendance():
    conn = sqlite3.connect("attendance.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name, date, time FROM attendance_log")
    records = cursor.fetchall()
    conn.close()
    
    if records:
        print("\nAttendance Log:")
        for record in records:
            print(f"Name: {record[0]}, Date: {record[1]}, Time: {record[2]}")
    else:
        print("No attendance records found")

# Main function
def main():
    global dataset_path
    dataset_path = "dataset/"
    
    # Initialize database
    init_db()
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    print("Face Recognition Attendance System")
    print("Press 'r' to register a new face, 'a' to mark attendance, 'v' to view attendance, 'q' to quit")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame")
            break
        
        # Display the frame
        cv2.imshow("Attendance System", frame)
        
        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('r'):
            name = input("Enter name for registration: ").strip()
            if name:
                register_face(name, frame)
                print(f"Registered {name}")
            else:
                print("Error: Name cannot be empty")
        
        elif key == ord('a'):
            result = recognize_and_log(frame)
            print(result)
        
        elif key == ord('v'):
            view_attendance()
        
        elif key == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Face Recognition Attendance System
Press 'r' to register a new face, 'a' to mark attendance, 'v' to view attendance, 'q' to quit
25-06-03 09:00:42 - Searching [[[ 99  99  99]
  [ 98  98  98]
  [ 97  97  97]
  ...
  [119 142 144]
  [119 142 144]
  [119 142 144]]

 [[100 100 100]
  [ 99  99  99]
  [ 97  97  97]
  ...
  [119 142 144]
  [118 141 143]
  [118 141 143]]

 [[100 100 100]
  [ 99  99  99]
  [ 98  98  98]
  ...
  [119 142 144]
  [118 141 143]
  [118 141 143]]

 ...

 [[106 136 137]
  [106 136 137]
  [107 137 138]
  ...
  [ 56  83  90]
  [ 55  82  89]
  [ 54  81  88]]

 [[104 135 136]
  [106 136 137]
  [107 137 138]
  ...
  [ 57  83  90]
  [ 56  82  89]
  [ 55  81  88]]

 [[104 136 136]
  [105 136 137]
  [107 137 138]
  ...
  [ 57  83  90]
  [ 56  82  89]
  [ 55  81  88]]] in 7 length datastore
25-06-03 09:00:44 - find function duration 2.4237987995147705 seconds
Attendance marked for shubh at 2025-06-03 09:00:44
25-06-03 09:00:55 - Searching [[[ 34  93 102]
  [ 35  94 103]
  [ 3

Enter name for registration:  aman


Registered aman at dataset//aman.jpg
Registered aman
25-06-03 09:01:20 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.07it/s]


25-06-03 09:01:21 - There are now 8 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
25-06-03 09:01:21 - Searching [[[ 32  92  98]
  [ 33  93  99]
  [ 33  93  99]
  ...
  [ 51 110 120]
  [ 50 109 119]
  [ 49 108 118]]

 [[ 33  92  98]
  [ 33  92  98]
  [ 34  93  99]
  ...
  [ 50 108 118]
  [ 50 108 118]
  [ 51 109 119]]

 [[ 35  92  98]
  [ 35  92  98]
  [ 36  93  99]
  ...
  [ 50 108 118]
  [ 50 108 117]
  [ 50 108 117]]

 ...

 [[ 53  39  41]
  [ 51  37  39]
  [ 49  35  37]
  ...
  [ 51  41  42]
  [ 51  41  41]
  [ 52  42  42]]

 [[ 50  36  38]
  [ 49  35  37]
  [ 48  34  36]
  ...
  [ 50  40  41]
  [ 51  41  41]
  [ 51  41  41]]

 [[ 49  35  37]
  [ 50  36  38]
  [ 50  36  38]
  ...
  [ 50  40  41]
  [ 50  40  40]
  [ 51  41  41]]] in 8 length datastore
25-06-03 09:01:21 - find function duration 0.41238880157470703 seconds
Attendance marked for aman at 2025-06-03 09:01:21
25-06-03 09:01:32 - Searching [[[ 80  83  84]
  [ 81  84  85]
  [ 82 